# Conway's Game of Life with PySAL

## Rules ##

   1. A cell that is currently alive and that has two **or** three live neighbors stays alive
   2. A cell that is currently dead with **exactly** three live neighbors comes alive 
   3. All other cells remain dead, or die due to loneliness (less than 2 neighbors) or overcrowding (more than 3 neighbors)

In [ ]:
import pysal as ps
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

from scipy.stats import bernoulli


In [ ]:
k = 8 # dimension of lattice

In [ ]:
w = ps.lat2W(k,k,rook=False)

In [ ]:
w.n

In [ ]:
w.neighbors[0]

In [ ]:
w.neighbors[45]

In [ ]:
w.weights[0]

In [ ]:
y = bernoulli.rvs(0.45,size=w.n)

In [ ]:
y

In [ ]:
wy = ps.lag_spatial(w,y)

In [ ]:
wy

## Rules ##

   1. A cell that is currently alive and that has two **or** three live neighbors stays alive
   2. A cell that is currently dead with **exactly** three live neighbors comes alive 
   3. All other cells remain dead, or die due to loneliness (less than 2 neighbors) or overcrowding (more than 3 neighbors)

Rule 1: find live cells and count their neighbors

In [ ]:
ywy = y*wy

In [ ]:
lw23 = np.nonzero( (ywy==2) + (ywy==3) )

In [ ]:
lw23

Rule 2: find dead cells with exactly 3 neighbors

In [ ]:
dw3 = (1-y) * wy

In [ ]:
np.nonzero(dw3==3)

Rules 1 and 2 give us the surviving cells

In [ ]:
live_next = np.nonzero( (ywy==2) + (ywy==3) + (dw3==3) )

In [ ]:
live_next

In [ ]:
y[live_next]

So we see that in the future, some dead cells will becoming alive. But what about live cells now that die in the next period?

We know that they will be dead next period. Allocate an array with zeros for the next period and assign the live cells. Everyone else is dead.

In [ ]:
y1 = np.zeros_like(y)

In [ ]:
y1[live_next] = 1

In [ ]:
y1

In [ ]:
def generation(y,w):
    y1 = np.zeros_like(y)
    wy = ps.lag_spatial(w,y)
    ywy = y * wy
    live_next = np.nonzero( ( ywy == 2 ) + ( ywy == 3 ) + ( ( 1-y ) * wy == 3 ) )
    y1[live_next] = 1
    return y1
        

In [ ]:
y = bernoulli.rvs(0.45,size=w.n)

In [ ]:
y1 = generation(y,w)

In [ ]:
y1

In [ ]:
y

In [ ]:
y2 = generation(y1,w)

In [ ]:
y2

One interesting initial pattern is the so called R-pentomino.

We will create one and then run a simulation to see how the solutions evolve.

In [ ]:
ngen=350
k = 50
w = ps.lat2W(k, k, rook=False)
#y = bernoulli.rvs(0.45,size=w.n)
y = np.zeros((w.n,))
#R-pentomino pattern
kd2 = k/2
top = kd2 +  k * ( kd2 - 1 )
topr = top + 1
midl = top + k -1
mid = midl + 1
bot = mid + k
y[[top, topr, midl, mid, bot]] = 1
results = {}
for i in xrange(ngen):
    y1 = generation(y,w)
    results[i] = y1
    if np.all(y == y1):
        break
    print i, y.sum(), y1.sum()
    y = y1
    

In [ ]:
generations = np.zeros((ngen,))
living = np.zeros_like(generations)
keys = results.keys()
keys.sort()
for i in keys:
    generations[i] = i
    living[i] = results[i].sum()
    if not i%10:
        ymat = results[i]
        ymat.shape = (50,50)
        plt.imshow(ymat,cmap='Greys', interpolation='nearest')
        plt.title("Generation %d"%i)
        plt.show()
    

In [ ]:
generations.shape

In [ ]:
plt.plot(generations,living)

In [ ]:
ymat = results[ngen-1]

In [ ]:
ymat.shape

In [ ]:
ymat.shape=(50,50)

In [ ]:
plt.imshow(ymat, cmap='Greys', interpolation='nearest')
plt.title("Last Generation")

In [ ]:
ymat = results[0]

In [ ]:
ymat.shape = (50,50)

In [ ]:
plt.imshow(ymat, cmap='Greys', interpolation='nearest')
plt.title('First Generation: R-pentomino')

In [ ]:
ymat = results[ngen-2]
ymat.shape=(50,50)
plt.imshow(ymat, cmap='Greys', interpolation='nearest')
plt.title("Penultimate Generation")

In [ ]:
ymat = results[ngen-1]
ymat.shape=(50,50)
plt.imshow(ymat, cmap='Greys', interpolation='nearest')
plt.title("Final Generation")